In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import albumentations as A
import numpy as np
from tqdm import tqdm
import shutil

# Define paths
IMG_DIR = "/content/drive/MyDrive/yolo_dataset/images-orignal/train"
LABEL_DIR = "/content/drive/MyDrive/yolo_dataset/labels-orignal/train"
OUT_IMG_DIR = "/content/drive/MyDrive/yolo_dataset4/images/train"
OUT_LABEL_DIR = "/content/drive/MyDrive/yolo_dataset4/labels/train"

os.makedirs(OUT_IMG_DIR, exist_ok=True)
os.makedirs(OUT_LABEL_DIR, exist_ok=True)

# Augmentations
AUGS = {
    "flip": A.Compose([A.HorizontalFlip(p=1.0)],
                      bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),
    "rotate_plus": A.Compose([A.Rotate(limit=(5, 5), p=1.0)],
                             bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),
    "rotate_minus": A.Compose([A.Rotate(limit=(-5, -5), p=1.0)],
                              bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),
    "scale": A.Compose([A.Affine(scale=(0.8, 0.8), p=1.0)],
                       bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))
}

img_files = [f for f in os.listdir(IMG_DIR) if f.endswith((".jpg", ".png"))]

def read_yolo_labels(label_path, img_w, img_h):
    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            cls, x, y, w, h = map(float, line.strip().split())
            x1 = (x - w / 2) * img_w
            y1 = (y - h / 2) * img_h
            x2 = (x + w / 2) * img_w
            y2 = (y + h / 2) * img_h
            bboxes.append([x1, y1, x2, y2, int(cls)])
    return bboxes

def save_yolo_labels(label_path, bboxes, img_w, img_h):
    with open(label_path, 'w') as f:
        for x1, y1, x2, y2, cls in bboxes:
            cx = ((x1 + x2) / 2) / img_w
            cy = ((y1 + y2) / 2) / img_h
            w = (x2 - x1) / img_w
            h = (y2 - y1) / img_h
            f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

for img_file in tqdm(img_files):
    img_path = os.path.join(IMG_DIR, img_file)
    label_path = os.path.join(LABEL_DIR, os.path.splitext(img_file)[0] + ".txt")

    image = cv2.imread(img_path)
    if image is None:
        continue
    height, width = image.shape[:2]
    bboxes = read_yolo_labels(label_path, width, height)

    if not bboxes:
        continue

    class0_boxes = [b for b in bboxes if b[4] == 0]
    class1_boxes = [b for b in bboxes if b[4] == 1]

    # Save original image and label regardless
    orig_img_out = os.path.join(OUT_IMG_DIR, img_file)
    orig_label_out = os.path.join(OUT_LABEL_DIR, os.path.splitext(img_file)[0] + ".txt")
    cv2.imwrite(orig_img_out, image)
    save_yolo_labels(orig_label_out, bboxes, width, height)

    # Apply augmentation only if class 1 boxes exist
    if class1_boxes:
        for aug_name, aug in AUGS.items():
            transformed = aug(
                image=image,
                bboxes=[b[:4] for b in class1_boxes],
                class_labels=[b[4] for b in class1_boxes]
            )

            aug_image = transformed["image"]
            aug_class1_bboxes = [
                list(map(float, bbox)) + [cls]
                for bbox, cls in zip(transformed["bboxes"], transformed["class_labels"])
            ]

            # Combine class 0 (unchanged) + transformed class 1
            all_bboxes = class0_boxes + aug_class1_bboxes

            # Save augmented image
            aug_img_name = f"{os.path.splitext(img_file)[0]}_{aug_name}.jpg"
            cv2.imwrite(os.path.join(OUT_IMG_DIR, aug_img_name), aug_image)

            # Save augmented labels
            aug_label_name = f"{os.path.splitext(img_file)[0]}_{aug_name}.txt"
            save_yolo_labels(
                os.path.join(OUT_LABEL_DIR, aug_label_name),
                all_bboxes,
                aug_image.shape[1],
                aug_image.shape[0]
            )


100%|██████████| 720/720 [00:48<00:00, 14.71it/s]


In [7]:
import os
from collections import defaultdict

# Path to the folder containing YOLOv8 .txt label files
LABEL_DIR = "/content/drive/MyDrive/yolo_dataset1/labels/train"

# Dictionary to hold class counts
class_counts = defaultdict(int)

# Loop through all .txt files
for file in os.listdir(LABEL_DIR):
    if file.endswith(".txt"):
        with open(os.path.join(LABEL_DIR, file), 'r') as f:
            for line in f:
                line = line.strip()
                if not line:  # Skip empty lines
                    continue

                # Skip lines that don't start with a number
                if not line[0].isdigit():
                    continue

                parts = line.split()
                try:
                    cls_id = int(float(parts[0]))
                    class_counts[cls_id] += 1
                except (ValueError, IndexError) as e:
                    print(f"Skipping malformed line in {file}: {line}")
                    continue

# Print the counts
print("Class counts:")
for cls_id in sorted(class_counts.keys()):
    print(f"Class {cls_id}: {class_counts[cls_id]}")

# Optional: Save to output file
output_path = os.path.join(LABEL_DIR, "class_counts.txt")
with open(output_path, 'w') as f:
    f.write("Class counts:\n")
    for cls_id in sorted(class_counts.keys()):
        f.write(f"Class {cls_id}: {class_counts[cls_id]}\n")

print(f"\nSaved class counts to {output_path}")

Class counts:
Class 0: 3960
Class 1: 4380

Saved class counts to /content/drive/MyDrive/yolo_dataset1/labels/train/class_counts.txt


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 853.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from ultralytics import YOLO
import yaml

# Load model
# model = YOLO("yolov8m.pt")

# Load YAML
with open("/content/drive/MyDrive/yolo_dataset4/hyp.yaml", 'r') as f:
    hyp_dict = yaml.safe_load(f)

# Train
model.train(
    data="/content/drive/MyDrive/yolo_dataset4/tb.yaml",
    epochs=500,
    imgsz=512,
    patience=50,
    workers=4,
    batch=16,
    project='/content/drive/MyDrive/yolo_models-enhance/YOLOv8m',
    **hyp_dict
)


Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.0777, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.265, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_dataset4/tb.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00941, lrf=0.0202, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.841, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=

train: Scanning /content/drive/MyDrive/yolo_dataset4/labels/train.cache... 1335 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1335/1335 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.6±0.5 ms, read: 19.4±9.6 MB/s, size: 68.8 KB)


val: Scanning /content/drive/MyDrive/yolo_dataset4/labels/val.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]


...

                   all         80        122      0.345      0.408      0.314      0.132
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 13, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



63 epochs completed in 0.575 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_models-enhance/YOLOv8m/train3/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/yolo_models-enhance/YOLOv8m/train3/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/yolo_models-enhance/YOLOv8m/train3/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.70it/s]


                   all         80        122      0.322      0.594      0.357      0.165
    ActiveTuberculosis         68        102      0.578      0.588      0.576       0.25
ObsoletePulmonaryTuberculosis         15         20     0.0651        0.6      0.139     0.0807
Speed: 0.4ms preprocess, 7.0ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_models-enhance/YOLOv8m/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x785cf8756950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804